In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [3]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from numpy.random import seed

from inception_model_generator import MRNet_inc_model

import tensorflow as tf
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc



Using TensorFlow backend.


#trainning on augmented balanced data with only the skewed class augmented
20 epochs


In [0]:
label_type = 'abnormal'
exam_view = 'coronal'
model, cbs = MRNet_inc_model(1,0.00001,combination=[label_type, exam_view])
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos

class_weight = {0: 1, 1: neg/pos}
if neg < pos:
  class_weight = {0: pos/neg, 1: 1}

print(class_weight)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l ,{0:1,1:1}, batch_size=1,model="inception",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
test_data_gen = MRNet_data_generator(data_path, p, l,{0:1,1:1} ,batch_size=1,model="inception",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)


model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.50.hdf5")
model.fit(x=train_data_gen, validation_data = test_data_gen,epochs=60, initial_epoch = 50,verbose=True, class_weight=class_weight,callbacks=[cbs[0]],shuffle=False, use_multiprocessing=True, workers=6)

no dropout
no dropout
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_2 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
{0: 4.242268041237113, 1: 1}
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  train
Combination:  abnormal  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/train
Number of inputs:  1017
input size:  (299, 299)
Initializing Data Generator:
factor :  1.0
model:  inception
data type:  test
Combination:  abnormal  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/test
Number of inputs:  113
input size:  (299, 299)
Epoch 51/60
1017/1017 [==============================] - 977s 961ms/step - loss: 0.4400

# **testing on validation dataset**

In [6]:
label_type = 'abnormal'
exam_view = 'coronal'
model, cbs = MRNet_inc_model(1,0,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_inception/"+label_type+"/"+exam_view+"/weights.55.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])

neg = total - pos
class_count_valid = {0: neg, 1: pos}

batch_size = 1
print(model.metrics_names)
valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=class_count_valid , batch_size=1,model="inception",data_type='valid', shuffle=False,label_type=label_type, exam_type=exam_view, scale_to=(299,299), aug_size=0)
d = model.evaluate_generator(valid_data_gen, total, verbose=1)
print(d)
model.predict_generator(valid_data_gen, 5, verbose=1)

no dropout
no dropout
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mr_net_inception_layer_3 (MR (None, 1)                 21804833  
Total params: 21,804,833
Trainable params: 21,770,401
Non-trainable params: 34,432
_________________________________________________________________
['loss', 'tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc']
Initializing Data Generator:
factor :  3.8
model:  inception
data type:  valid
Combination:  abnormal  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (299, 299)
120/120 [==============================] - 17s 141ms/step
[0.21459808945655823, 42.75833511352539, 6.891666889190674, 5.641666889190674, 5.208333492279053, 0.7846594452857971, 0.8298147320747375, 0.9099451899528503, 0.6909539699554443]
5/5 [==============================] - 3s 537ms/ste

array([[0.935527  ],
       [0.97845876],
       [0.57160026],
       [0.966437  ],
       [0.6526432 ]], dtype=float32)

In [0]:
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  y_true = []
  y_score = []
  for i in range(len(valid_data_gen)):
    x, y = next(valid_data_gen)
    _y = model.predict_classes(x, batch_size=1, verbose=0)
    _y_score = model.predict_proba(x, batch_size=1)
    y_score.append(_y_score[0][0])
    y_true.append(y[0])
    if _y[0][0] == 1:
      if _y[0][0] == y[0]:
        tp += 1
      else:
        fp += 1
    else:
      if _y[0][0] == y[0]:
        tn += 1
      else:
        fn += 1
  precision = tp / (tp +fp)
  recall = tp / (tp + fn)
  f1 = 2*((precision*recall)/(precision+recall))
  y_score = np.array(y_score)
  y_true = np.array(y_true)
  fpr, tpr, _ = roc_curve(y_true, y_score)
  auc = roc_auc(fpr, tpr)
  print ("tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " F1 = ", f1, " auc = ", auc)

tp =  82  fp =  14  tn =  11  fn =  13  accuracy =  0.775  F1 =  0.8586387434554975  auc =  0.7187368421052631
